In [1]:
import mlflow
mlflow.set_tracking_uri("http://ec2-18-233-10-235.compute-1.amazonaws.com:5000/")

In [2]:
mlflow.set_experiment("ML Algos with HP Tuning")

<Experiment: artifact_location='s3://ms-mlflow-bucket/580262608986777681', creation_time=1751096624948, experiment_id='580262608986777681', last_update_time=1751096624948, lifecycle_stage='active', name='ML Algos with HP Tuning', tags={}>

In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
import mlflow
import mlflow.sklearn
import optuna

c:\Users\miles\Desktop\DSMP\Projects\yt-sentiment-analysis\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df = pd.read_csv('data_preprocessed.csv').dropna().drop_duplicates()
df.shape

(36243, 2)

In [5]:
# Step 1: (Optional) Remapping - skipped since not strictly needed for SVM

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

# Step 3: TF-IDF vectorizer setup
ngram_range = (1, 3)  # Trigram
max_features = 2000  
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X = vectorizer.fit_transform(df['clean_comment'])
y = df['category']

# Step 4: Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_2000_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Step 6: Optuna objective function for SVM
def objective_svm(trial):
    C = trial.suggest_float('C', 1e-4, 10.0, log=True)
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly'])

    model = SVC(C=C, kernel=kernel, random_state=42)
    return accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test))


# Step 7: Run Optuna for SVM, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_svm, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = SVC(C=best_params['C'], kernel=best_params['kernel'], random_state=42)

    # Log the best model with MLflow, passing the algo_name as "SVM"
    log_mlflow("SVM", best_model, X_train, X_test, y_train, y_test)

# Run the experiment for SVM
run_optuna_experiment()


[I 2025-06-28 13:19:00,362] A new study created in memory with name: no-name-e3fe4faf-f376-40b3-bd6e-4df47d08e059
[I 2025-06-28 13:24:42,267] Trial 0 finished with value: 0.414468085106383 and parameters: {'C': 0.002091930318208018, 'kernel': 'poly'}. Best is trial 0 with value: 0.414468085106383.
[I 2025-06-28 13:29:32,102] Trial 1 finished with value: 0.7801063829787234 and parameters: {'C': 0.1837561888974706, 'kernel': 'rbf'}. Best is trial 1 with value: 0.7801063829787234.
[I 2025-06-28 13:34:13,828] Trial 2 finished with value: 0.41510638297872343 and parameters: {'C': 0.0027716799718390515, 'kernel': 'rbf'}. Best is trial 1 with value: 0.7801063829787234.
[I 2025-06-28 13:38:33,401] Trial 3 finished with value: 0.5659574468085107 and parameters: {'C': 0.006300118192423408, 'kernel': 'linear'}. Best is trial 1 with value: 0.7801063829787234.
[I 2025-06-28 13:42:53,754] Trial 4 finished with value: 0.414468085106383 and parameters: {'C': 0.0004994319516645163, 'kernel': 'poly'}. B

🏃 View run SVM_2000_SMOTE_TFIDF_Trigrams at: http://ec2-18-233-10-235.compute-1.amazonaws.com:5000/#/experiments/580262608986777681/runs/1201ed89f0f8427c900ccc4d80ccdd12
🧪 View experiment at: http://ec2-18-233-10-235.compute-1.amazonaws.com:5000/#/experiments/580262608986777681
